# Seperability Insight into OPT models
Tests to see if it is possible to remove coding ability from Meta OPT model for different scales.
Current methods are:
- look at activation frequency of MLP mid layers
- Look at 'crossover threshold' of Attention heads

In [ ]:
try: # if in google colab, download necessary python files
  import google.colab 
  ! git clone https://github.com/pesvut/opt-tools.git && mv ./opt-tools/src/*.py .
except ModuleNotFoundError:
  pass
! pip3 install -qqr requirements.txt 

In [ ]:
import torch
import numpy as np
import pandas as pd
import einops
import matplotlib.pyplot as plt
import wandb

from model import Model
from texts import prepare
from activations import prune_and_evaluate, evaluate_all, init_data_dict

# df.append has FutureWarning, here is the alternative
def append( df: pd.DataFrame, data: dict ):
    wandb.log(data)
    data = pd.DataFrame({ k:[v] for k,v in data.items() })
    return pd.concat([ df, data ], ignore_index=True )

In [ ]:
# Configure initial model and tests
model_size, token_limit  = "125m", 1000
run_pre_test             = True
pre_removals = []

# Removals parameters
ff_frac,   ff_eps   = 0.02, 0.001
attn_frac           = 0.01
cripple, focus      = "code", "pile"



In [ ]:
# Prepare data logging
wandb.init(project="seperability-code-pile")
df = pd.DataFrame()

# Load model and show details about model
opt = Model( model_size, limit=token_limit )

# Pre-pruning of model
opt.delete_ff_keys_from_files(pre_removals)

In [ ]:
# Evaluate model before removal of any neurons
if run_pre_test:
    data = init_data_dict()
    data.update( evaluate_all( opt, 1e4 ) )
    df = append( df, data )

In [ ]:
# First do some pruning of the feed forward layers
for i in range(3):
    data = prune_and_evaluate( opt, ff_frac, 0, ff_eps, cripple=cripple, focus=focus )
    df = append( df, data )
print(df.T)

In [ ]:
# Next do some pruning of the feed forward layers + attention layers
for i in range(3):
    data = prune_and_evaluate( opt, ff_frac, attn_frac, ff_eps, cripple=cripple, focus=focus )
    df = append( df, data )
print(df.T)